halving

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {
    "model__n_estimators": [50, 100, 200, 500],
    "model__max_depth": [None, 10, 20, 30],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 5, 10],
    "model__max_features": ["sqrt", "log2", None],
    "model__bootstrap": [True, False],
    "model__class_weight": ["balanced", "balanced_subsample", None]
}

halving_search = HalvingGridSearchCV(
    base_model_pipe,    # Model pipeline
    param_grid,     # Gunakan param_grid yang sama
    factor = 3,     # Kandidat dikurangi 3x di setiap iterasi
    cv = 3,     # Cross-validation lebih kecil untuk kecepatan
    scoring = ["precision", "recall", "f1"],    # Metrik utama tetap recall
    refit = "recall", 
    n_jobs = -1,    # Gunakan semua core CPU
    verbose = 2     # Tampilkan progress
)

halving_search.fit(X_train, y_train)

best_model = halving_search.best_estimator_

In [ ]:
# Pilih model terbaik berdasarkan akurasi
halving_results = halving_search.cv_results_
halving_results.keys()

print("Best parameters based on recall:", halving_search.best_params_)

In [ ]:
# Cek metrik lainnya jika menggunakan multiple scoring
print("Best model based on precision:", np.max(halving_results['mean_test_precision']))
print("Best model based on F1-score:", np.max(halving_results['mean_test_f1']))

In [ ]:
# Evaluasi Model Terbaik
y_pred_best = best_model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred_best))

Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     55748
         1.0       1.00      1.00      1.00       995

    accuracy                           1.00     56743
   macro avg       1.00      1.00      1.00     56743
weighted avg       1.00      1.00      1.00     56743

Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
AUC-ROC Score: 1.0000


In [ ]:
# Metrik tambahan
accuracy = accuracy_score(y_test, y_pred_best)
precision = precision_score(y_test, y_pred_best)
recall = recall_score(y_test, y_pred_best)
f1 = f1_score(y_test, y_pred_best)
auc_score = roc_auc_score(y_test, y_pred_best)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC Score: {auc_score:.4f}")